In [1]:
from path import Path
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

In [3]:
path = Path('../input')

### Datos de entrenamiento

In [4]:
train_df      = pd.read_csv(path/'train.csv')
train_weather = pd.read_csv(path/"weather_train.csv")

### Datos de test

In [5]:
test_df       = pd.read_csv(path/'test.csv')
test_weather  = pd.read_csv(path/"weather_test.csv")

### Edificios

In [6]:
building_df   = pd.read_csv(path/'building_metadata.csv')

### Opcional: Reducir memoria

In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

train_weather = reduce_mem_usage(train_weather)
test_weather  = reduce_mem_usage(test_weather)

building_df = reduce_mem_usage(building_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


### Metemos las columnas de meta-datos de edificios y climatología

In [9]:
train_df = train_df.merge(building_df, on='building_id', how='left')
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN


In [10]:
train_df = train_df.merge(train_weather, on=['site_id', 'timestamp'], how='left')
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.000000,0,Education,7432,2008.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
1,1,0,2016-01-01 00:00:00,0.000000,0,Education,2720,2004.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
2,2,0,2016-01-01 00:00:00,0.000000,0,Education,5376,1991.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
3,3,0,2016-01-01 00:00:00,0.000000,0,Education,23685,2002.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
4,4,0,2016-01-01 00:00:00,0.000000,0,Education,116607,1975.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
5,5,0,2016-01-01 00:00:00,0.000000,0,Education,8000,2000.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
6,6,0,2016-01-01 00:00:00,0.000000,0,Lodging/residential,27926,1981.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
7,7,0,2016-01-01 00:00:00,0.000000,0,Education,121074,1989.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
8,8,0,2016-01-01 00:00:00,0.000000,0,Education,60809,2003.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
9,9,0,2016-01-01 00:00:00,0.000000,0,Office,27000,2010.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000


### Qué atributos vamos a usar

In [11]:
features = ['square_feet', 'year_built', 'air_temperature', 'dew_temperature', 
            'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr', 
            'floor_count']

### Definimos la x (entrada) y la y (salida o valor a predecir)

In [12]:
x = train_df[features].values
y = np.log1p(train_df[['meter_reading']].values.reshape(-1,1))

### Qué pasa con los NaNs?

In [14]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(x)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

### Ejercicio: ver si hay NaNs antes de imputarlos

In [16]:
x = imp.transform(x)

### Ejercicio: ver si hay NaNs después de imputarlos

### Regresión lineal

In [18]:
clf = LinearRegression()
clf.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Metemos las columnas de meta-datos de edificios y climatología (en test)

In [20]:
test_df = test_df.merge(building_df, on='building_id', how='left')
test_df = test_df.merge(test_weather, on=['site_id', 'timestamp'], how='left')
test_df.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609


In [21]:
test = test_df[features].values
test

array([[7.43200000e+03, 2.00800000e+03, 1.77968750e+01, ...,
        3.59960938e+00,            nan,            nan],
       [2.72000000e+03, 2.00400000e+03, 1.77968750e+01, ...,
        3.59960938e+00,            nan,            nan],
       [5.37600000e+03, 1.99100000e+03, 1.77968750e+01, ...,
        3.59960938e+00,            nan,            nan],
       ...,
       [1.12650000e+04, 1.99700000e+03,            nan, ...,
                   nan,            nan,            nan],
       [2.97750000e+04, 2.00100000e+03,            nan, ...,
                   nan,            nan,            nan],
       [9.22710000e+04, 2.00100000e+03,            nan, ...,
                   nan,            nan,            nan]])

In [22]:
test = imp.transform(test)
test

array([[7.43200000e+03, 2.00800000e+03, 1.77968750e+01, ...,
        3.59960938e+00, 7.96415539e-01, 4.18484813e+00],
       [2.72000000e+03, 2.00400000e+03, 1.77968750e+01, ...,
        3.59960938e+00, 7.96415539e-01, 4.18484813e+00],
       [5.37600000e+03, 1.99100000e+03, 1.77968750e+01, ...,
        3.59960938e+00, 7.96415539e-01, 4.18484813e+00],
       ...,
       [1.12650000e+04, 1.99700000e+03, 1.59879721e+01, ...,
        3.37750300e+00, 7.96415539e-01, 4.18484813e+00],
       [2.97750000e+04, 2.00100000e+03, 1.59879721e+01, ...,
        3.37750300e+00, 7.96415539e-01, 4.18484813e+00],
       [9.22710000e+04, 2.00100000e+03, 1.59879721e+01, ...,
        3.37750300e+00, 7.96415539e-01, 4.18484813e+00]])

### Predicciones

In [23]:
preds = np.expm1(clf.predict(test))
preds

array([[37.41387549],
       [35.73475151],
       [34.67226967],
       ...,
       [36.55855488],
       [41.93112738],
       [63.24548741]])

### Pregunta: ¿Por qué expm1?

### Preparemos el envío

In [24]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [27]:
sample_submission['meter_reading'] = preds
sample_submission.head()

,row_id,meter_reading
0,0,37.413875
1,1,35.734752
2,2,34.672270
3,3,40.754263
4,4,68.036311


### A disco en un CSV de toda la vida

In [30]:
csv_fname = '1-regresion-lineal.csv.zip'
sample_submission.to_csv(csv_fname, compression='zip', index=False)

### Opcional: enviar a Kaggle (OJO! que hay limite diario)

In [ ]:
comp = 'ashrae-energy-prediction'
!kaggle competitions submit -c {comp} -f {csv_fname}-m 'regresión lineal simplona'